In [1]:
from ratelimit import limits, sleep_and_retry

@sleep_and_retry
@limits(calls=1, period=1)
def ratelimit():
    return


In [2]:
import requests
import json
from pydub import AudioSegment

In [5]:
def api_request(query):
    ratelimit()
    r = requests.get(f"https://xeno-canto.org/api/2/recordings?query={query}")
    return r.json()


def get_recordings(query):
    recordings = []
    r = api_request(query)
    numpages = r["numPages"]
    for i in range(1,numpages+1):
        if i > 1:
            r = api_request(query + f"&page={i}")
        recordings += r["recordings"]
    return recordings
        
def download_recording(recording, folder):
    with open(f"{folder}/{recording["id"]}.json", "wt") as f:
        json.dump(recording,f, indent=2)

    ratelimit()
    r = requests.get(recording["file"])
    with open(f"{folder}/{recording["id"]}.wav", "wb") as f:
        f.write(r.content)

    filetype = recording["file-name"].strip()
    if not filetype.endswith(".wav"):
        sound = AudioSegment.from_file(f"{folder}/{recording["id"]}.wav")
        sound.export(f"{folder}/{recording["id"]}.wav", format="wav")
        
def download_recordings(recordings, folder):
    for r in recordings:
        download_recording(r, folder)


In [6]:

recordings = get_recordings("great tit q:A")

KeyError: 'numPages'

In [ ]:
len(recordings)

download_recordings(recordings, "data/downloaded")
